In [3]:
import pandas as pd
import requests
import json
import os
import csv
from pprint import pprint

In [ ]:
#listing API urls to use in data pulls
base_url = 'https://public.opendatasoft.com/api/records/1.0/'
records_url_listings = 'search/?dataset=airbnb-listings'
download_url_listings = 'download/?dataset=airbnb-listings'
records_url_reviews = 'search/?dataset=airbnb-reviews'
download_url_reviews = 'download/?dataset=airbnb-reviews'